In [1]:
import numpy as np
from scipy.spatial import distance_matrix
import pandas as pd
import pickle
import re
import os

from pyeer.cmc_stats import load_scores_from_file, get_cmc_curve, CMCstats
from pyeer.report import generate_cmc_report
from pyeer.plot import plot_cmc_stats

import shutil

In [2]:
#Nos aseguramos de usar GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
!pip install -U scipy==1.2.0
!git clone https://github.com/michuanhaohao/AlignedReID.git

Requirement already up-to-date: scipy==1.2.0 in c:\users\usuario\anaconda3\envs\entornogpu\lib\site-packages (1.2.0)


Cloning into 'AlignedReID'...


In [3]:
%cd ./AlignedReID

C:\Users\Usuario\OneDrive\Documentos\Master\TFM\TFM\AlignedReID


In [5]:
!mkdir data #En esta carpeta poner el dataset
!mkdir embedding
!mkdir matrices
!mkdir CMC

In [11]:
!python train_alignedreid.py -d market1501 -a resnet50 --test_distance global_local --reranking

Args:Namespace(arch='resnet50', cuhk03_classic_split=False, cuhk03_labeled=False, dataset='market1501', eval_step=-1, evaluate=False, gamma=0.1, gpu_devices='0', height=256, htri_only=False, labelsmooth=False, lr=0.0002, margin=0.3, max_epoch=300, num_instances=4, optim='adam', print_freq=10, reranking=True, resume='', root='data', save_dir='log', seed=1, split_id=0, start_epoch=0, start_eval=0, stepsize=150, test_batch=32, test_distance='global_local', train_batch=32, unaligned=False, use_cpu=False, use_metric_cuhk03=False, weight_decay=0.0005, width=128, workers=4)
Currently using GPU 0
Initializing dataset market1501
=> Market1501 loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # images
  ------------------------------
  train    |   600 |     3255
  query    |   750 |     3368
  gallery  |   442 |      813
  ------------------------------
  total    |  1350 |     7436
  ------------------------------
Initializing model: resnet50
Model size: 25.00380

C:\Users\Usuario\OneDrive\Documentos\Master\TFM\TFM\AlignedReID\util\losses.py:151: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  dist.addmm_(1, -2, inputs, inputs.t())


In [14]:
!python train_alignedreid.py -d market1501 -a resnet50 --evaluate --resume "./log/best_model.pth.tar" --save-dir log/resnet50-cuhk03-alignedreid --test_distance global_local

Args:Namespace(arch='resnet50', cuhk03_classic_split=False, cuhk03_labeled=False, dataset='market1501', eval_step=-1, evaluate=True, gamma=0.1, gpu_devices='0', height=256, htri_only=False, labelsmooth=False, lr=0.0002, margin=0.3, max_epoch=300, num_instances=4, optim='adam', print_freq=10, reranking=False, resume='./log/best_model.pth.tar', root='data', save_dir='log/resnet50-cuhk03-alignedreid', seed=1, split_id=0, start_epoch=0, start_eval=0, stepsize=150, test_batch=32, test_distance='global_local', train_batch=32, unaligned=False, use_cpu=False, use_metric_cuhk03=False, weight_decay=0.0005, width=128, workers=4)
Currently using GPU 0
Initializing dataset market1501
=> Market1501 loaded
Dataset statistics:
  ------------------------------
  subset   | # ids | # images
  ------------------------------
  train    |   600 |     3255
  query    |   750 |     3368
  gallery  |   442 |      813
  ------------------------------
  total    |  1350 |     7436
  ----------------------------

train_alignedreid.py:329: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:882.)
  distmat.addmm_(1, -2, qf, gf.t())


In [16]:
!python GenerateBodyEmbeddings.py ".\\imagenes_5Cams" ".\\embedding" Market1501

In [17]:
#La funcion debe meterle los ficheros de cada camara y devolver una matriz que sea "foto/dorsales"
def Matrix(datos1,datos2):
    dorsales = []
    num_img = []
    matrix =[]
    
    for x in range(len(datos1)):
        fila = []
        
        for y in range(len(datos2)):
            #### REVISAR ###########################
            dist_mat = distance_matrix(datos1[x]["embedding"], datos2[y]["embedding"], p=2)
            dist_mat = sum(np.diag(dist_mat))
            fila.append(dist_mat)
            
        matrix.append(fila)
    
    for x in range(len(datos1)):
        text = 'dorsal_' + str(datos1[x]['bib']) + '_imagen_' + str(x+1)
        num_img.append(text)
        
    for x in range(len(datos2)):
        text = 'dorsal_' + str(datos2[x]['bib']) + '_tiempo_' + str(datos2[x]['time'])
        dorsales.append(text)
        
    df = pd.DataFrame(matrix, columns=dorsales, index=num_img)
    
    return df

In [21]:
with open('.\\embedding\\imagenes_5Cams.pkl', 'rb') as f:
    data = pickle.load(f)

data_c1s1 = []
data_c2s1 = []
data_c3s1 = []
data_c4s1 = []
data_c5s1 = []

for image in range(len(data)):
    if data[image]["rp"] == 'c1s1':
        data_c1s1.append(data[image])
        
    elif data[image]["rp"] == 'c2s1':
        data_c2s1.append(data[image])
        
    elif data[image]["rp"] == 'c3s1':
        data_c3s1.append(data[image])
        
    elif data[image]["rp"] == 'c4s1':
        data_c4s1.append(data[image])
        
    elif data[image]["rp"] == 'c5s1':
        data_c5s1.append(data[image])

In [24]:
c1s1_c2s1 = Matrix(data_c1s1,data_c2s1)
c1s1_c3s1 = Matrix(data_c1s1,data_c3s1)
c1s1_c4s1 = Matrix(data_c1s1,data_c4s1)
c1s1_c5s1 = Matrix(data_c1s1,data_c5s1)

c2s1_c1s1 = Matrix(data_c2s1,data_c1s1)
c2s1_c3s1 = Matrix(data_c2s1,data_c3s1)
c2s1_c4s1 = Matrix(data_c2s1,data_c4s1)
c2s1_c5s1 = Matrix(data_c2s1,data_c5s1)

c3s1_c1s1 = Matrix(data_c3s1,data_c1s1)
c3s1_c2s1 = Matrix(data_c3s1,data_c2s1)
c3s1_c4s1 = Matrix(data_c3s1,data_c4s1)
c3s1_c5s1 = Matrix(data_c3s1,data_c5s1)

c4s1_c1s1 = Matrix(data_c4s1,data_c1s1)
c4s1_c2s1 = Matrix(data_c4s1,data_c2s1)
c4s1_c3s1 = Matrix(data_c4s1,data_c3s1)
c4s1_c5s1 = Matrix(data_c4s1,data_c5s1)

c5s1_c1s1 = Matrix(data_c5s1,data_c1s1)
c5s1_c2s1 = Matrix(data_c5s1,data_c2s1)
c5s1_c3s1 = Matrix(data_c5s1,data_c3s1)
c5s1_c4s1 = Matrix(data_c5s1,data_c4s1)

c1s1_c2s1.to_csv(r'.\matrices\c1s1_c2s1.csv', index = True)
c1s1_c3s1.to_csv(r'.\matrices\c1s1_c3s1.csv', index = True)
c1s1_c4s1.to_csv(r'.\matrices\c1s1_c4s1.csv', index = True)
c1s1_c5s1.to_csv(r'.\matrices\c1s1_c5s1.csv', index = True)

c2s1_c1s1.to_csv(r'.\matrices\c2s1_c1s1.csv', index = True)
c2s1_c3s1.to_csv(r'.\matrices\c2s1_c3s1.csv', index = True)
c2s1_c4s1.to_csv(r'.\matrices\c2s1_c4s1.csv', index = True)
c2s1_c5s1.to_csv(r'.\matrices\c2s1_c5s1.csv', index = True)

c3s1_c1s1.to_csv(r'.\matrices\c3s1_c1s1.csv', index = True)
c3s1_c2s1.to_csv(r'.\matrices\c3s1_c2s1.csv', index = True)
c3s1_c4s1.to_csv(r'.\matrices\c3s1_c4s1.csv', index = True)
c3s1_c5s1.to_csv(r'.\matrices\c3s1_c5s1.csv', index = True)

c4s1_c1s1.to_csv(r'.\matrices\c4s1_c1s1.csv', index = True)
c4s1_c2s1.to_csv(r'.\matrices\c4s1_c2s1.csv', index = True)
c4s1_c3s1.to_csv(r'.\matrices\c4s1_c3s1.csv', index = True)
c4s1_c5s1.to_csv(r'.\matrices\c4s1_c5s1.csv', index = True)

c5s1_c1s1.to_csv(r'.\matrices\c5s1_c1s1.csv', index = True)
c5s1_c2s1.to_csv(r'.\matrices\c5s1_c2s1.csv', index = True)
c5s1_c3s1.to_csv(r'.\matrices\c5s1_c3s1.csv', index = True)
c5s1_c4s1.to_csv(r'.\matrices\c5s1_c4s1.csv', index = True)

In [25]:
c1s1_c2s1

,dorsal_1_tiempo_1:17:16,dorsal_2_tiempo_1:35:18,dorsal_5_tiempo_1:54:23,dorsal_6_tiempo_1:17:23,dorsal_7_tiempo_1:41:20,dorsal_9_tiempo_1:36:01,dorsal_10_tiempo_1:21:41,dorsal_10_tiempo_1:21:42,dorsal_11_tiempo_1:59:24,dorsal_12_tiempo_2:07:05,...,dorsal_795_tiempo_2:19:29,dorsal_795_tiempo_2:19:31,dorsal_801_tiempo_2:35:42,dorsal_801_tiempo_2:35:43,dorsal_804_tiempo_2:11:05,dorsal_804_tiempo_2:11:06,dorsal_812_tiempo_2:06:06,dorsal_812_tiempo_2:06:07,dorsal_813_tiempo_1:47:49,dorsal_813_tiempo_1:47:50
dorsal_1_imagen_1,3.593176,7.499981,8.843398,8.371540,8.386703,8.746177,7.395879,7.086861,8.290539,8.221777,...,8.295898,8.859904,8.343392,8.812037,9.581058,9.563217,8.692705,8.543884,8.077738,7.642407
dorsal_1_imagen_2,3.481657,7.969557,9.129650,8.341377,8.562712,8.967796,7.012649,6.886351,8.578359,8.608540,...,8.388014,8.770028,7.788847,8.052810,9.673177,9.654619,8.947909,8.622818,7.911537,7.921116
dorsal_2_imagen_3,6.720877,3.479059,8.187446,6.989969,6.491866,8.527376,7.922967,7.738907,7.495868,8.024743,...,8.621375,8.976753,8.313928,8.689963,8.750304,8.508291,8.147633,7.750110,8.027426,7.721227
dorsal_2_imagen_4,7.174934,3.919072,7.904309,7.145965,6.778893,8.505310,8.305270,8.186651,7.607420,8.010015,...,8.408721,8.531687,8.211585,8.493908,8.455681,8.298524,8.461239,7.890178,7.633963,7.644811
dorsal_5_imagen_5,8.736513,8.097304,3.522159,7.835937,7.852057,8.200653,9.159961,8.888008,7.029131,7.909305,...,6.978019,8.428449,8.915897,9.513021,7.116706,6.609297,6.840532,6.638656,9.317376,9.331380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dorsal_812_imagen_385,7.685584,8.126686,7.351770,7.051580,7.581047,8.911056,8.899994,8.254376,8.513294,8.464584,...,7.814755,8.997148,8.648885,8.812361,8.372582,7.545654,4.891743,4.717042,9.222021,9.186332
dorsal_813_imagen_386,7.898898,7.905103,8.925506,9.020991,9.194842,7.967193,7.098032,7.855092,7.370780,8.653964,...,8.723150,8.206862,8.708764,8.319853,8.752687,9.508008,9.476326,9.246067,2.724157,3.577745
dorsal_813_imagen_387,7.553233,7.879638,8.841906,8.856264,9.099059,8.204190,7.312655,7.928391,7.545707,8.510732,...,8.690537,8.248304,8.531681,8.294795,8.686918,9.526717,9.391176,9.201684,3.432709,4.035924
dorsal_813_imagen_388,7.873304,7.964502,9.001425,9.065519,9.029796,7.652221,7.021249,7.801704,7.635143,8.581522,...,8.900465,8.389626,8.852928,8.415017,8.662526,9.507240,9.346815,9.180657,3.204221,3.794749


In [26]:
!pip install pyeer

In [28]:
def data_scores(data, folder):
    file = folder + '/' + 'data_scores.txt'
    with open(file, 'w') as f:
        for index in data.index:
            for col in data.columns:
                if col == 'Unnamed: 0':
                    continue

                value = data.at[index,col]

                f.write(data.at[index,'Unnamed: 0'])
                f.write(' ')
                f.write(col)
                f.write(' ')
                f.write(str( (1/value) ) )
                f.write('\n')

In [22]:
def data_tp(data, folder):
    file = folder + '/' + 'data_tp.txt'
    with open(file, 'w') as f:
        for index in data.index:
            for col in data.columns:
                if col == 'Unnamed: 0':
                    continue

                dorsal_index = data.at[index,'Unnamed: 0']
                dorsal_col = col

                match_index = re.search("^(dorsal_\d+)",dorsal_index)
                match_col = re.search("^(dorsal_\d+)",dorsal_col)

                if str(match_index.group(1)) == str(match_col.group(1)):
                    f.write(dorsal_index)
                    f.write(' ')
                    f.write(col)
                    f.write('\n')

In [9]:
def CMC(data_scores, data_tp, folder, rank):
    # CMC maximum rank  Original:20
    r = rank

    # Loading scores
    sfile = data_scores  # The scores file
    tp_file = data_tp  # The genuine pairs relationship in "sfile"
    scores = load_scores_from_file(sfile, tp_file)

    # Calculating CMC curve
    ranks = get_cmc_curve(scores, r)

    # Creating stats
    stats = [CMCstats(exp_id='A', ranks=ranks)]

    # Generating CSV report
    DIR = folder + '/pyeer_report.csv' 
    generate_cmc_report(stats, r, DIR)

    # Generating HTML report
    DIR = folder + '/pyeer_report.html' 
    generate_cmc_report(stats, r, DIR)

    # Generating Latex report
    DIR = folder + '/pyeer_report.tex'
    generate_cmc_report(stats, r, DIR)

    # Generating Latex report
    DIR = folder + '/pyeer_report.json'
    generate_cmc_report(stats, r, DIR)

    # Plotting
    plot_cmc_stats(stats, r)
    
    delete_file = folder + '/CMC.png'
    if os.path.exists(delete_file):
            os.remove(delete_file)
    
    shutil.move('./CMC.png', folder)

In [7]:
%cd ./CMC

C:\Users\Usuario\OneDrive\Documentos\Master\TFM\TFM\AlignedReID\CMC


In [29]:
CamarasEnfrentadas = ['c1s1_c2s1', 'c2s1_c3s1', 'c3s1_c4s1', 'c4s1_c5s1', 'c5s1_c4s1']

for x in CamarasEnfrentadas:
    
    dir_data = './../matrices/'+ x +'.csv'
    data = pd.read_csv(dir_data)
    folder = os.path.join('./', x)
    data_scores_dir = folder + '/data_scores.txt'
    data_tp_dir = folder + '/data_tp.txt' 
    
    data_scores(data, folder)
    data_tp(data, folder)
    CMC(data_scores_dir, data_tp_dir, folder, 20)

In [13]:
!getcmcinf -h

usage: getcmcinf [-h] [-p PATH] -ms SCORES_FILENAMES -t TRUE_PAIRS_FILE_NAMES
                 -e EXPERIMENT_NAMES [-r MAXIMUM_RANK] [-lw LINE_WIDTH]
                 [-lf LEGEND_FONT] [-np] [-sp SAVE_PATH] [-pf PLOTS_FORMAT]
                 [-rf REPORT_FORMAT] [-sr SAVE_DPI] [-ds]

optional arguments:
  -h, --help            show this help message and exit
  -p PATH, --path PATH  The path to the scores files
  -ms SCORES_FILENAMES, --scores_filenames SCORES_FILENAMES
                        The scores file. Multiple files must be separated by a
                        comma
  -t TRUE_PAIRS_FILE_NAMES, --true_pairs_file_names TRUE_PAIRS_FILE_NAMES
                        Genuine pairs file. Multiple files must be separated
                        by a comma.
  -e EXPERIMENT_NAMES, --experiment_names EXPERIMENT_NAMES
                        Experiment ID. Multiple IDS must be separated by comma
  -r MAXIMUM_RANK, --maximum_rank MAXIMUM_RANK
                        The maximum rank to c